### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
import os
import pandas as pd
import json

In [2]:
from dotenv import load_dotenv

In [3]:
# Set environment variables from the .env in the local environment
load_dotenv()

api_key = os.getenv('NYT_API_KEY')
type(api_key)


str

### Access the New York Times API

In [4]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL

search_url = f"{url}api-key={api_key}&begin_date={begin_date}&end_date={end_date}"+ f'&fq={filter_query}&sort={sort}&fl={field_list}'

requests.get(search_url)

<Response [200]>

In [5]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0,20):

    # create query with a page number
    query_page_num = f'{search_url}&page={page}'

    # API results show 10 articles at a time (no code here)

    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_page_num).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    
    time.sleep(12)

    
    # Try and save the reviews to the reviews_list
    
    try:

        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews['response']['docs']:
            reviews_list.append(review)

        # Print the page that was just retrieved
        print(f'Page {page} has been read. ')
    except:

        # Print the page number that had no results then break from the loop
        print('There were no results for {page}. ')

Page 0 has been read. 
Page 1 has been read. 
Page 2 has been read. 
Page 3 has been read. 
Page 4 has been read. 
Page 5 has been read. 
Page 6 has been read. 
Page 7 has been read. 
Page 8 has been read. 
Page 9 has been read. 
Page 10 has been read. 
Page 11 has been read. 
Page 12 has been read. 
Page 13 has been read. 
Page 14 has been read. 
Page 15 has been read. 
Page 16 has been read. 
Page 17 has been read. 
Page 18 has been read. 
Page 19 has been read. 


In [6]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

print(json.dumps(reviews_list[0:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [58]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()

reviews_df = pd.json_normalize(reviews_list)
reviews_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [59]:
reviews_df.columns

Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.person',
       'byline.organization'],
      dtype='object')

In [60]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"

# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [61]:
# Extract 'name' and 'value' from items in "keywords" column

def extract_keywords(keyword_list):
    if isinstance(keyword_list, list): #THIS IS ALL chatGpt
        extracted_keywords = ""
        for item in keyword_list:
            if isinstance(item, dict) and 'name' in item and 'value' in item: #ChatGPT code assist
                # Extract 'name' and 'value'
                keyword = f"{item['name']}: {item['value']}; "
                # Append the keyword item to the extracted_keywords string
                extracted_keywords += keyword
        return extracted_keywords.strip() #the .strip was ChatGPT - I was getting a "string indices must be integers error.
    return ""

#Use the supplied extract_keywords function to convert the "keywords" column from 
# a list of dictionaries to strings using the apply() method.

# Fix the "keywords" column by converting cells from a list to a string - DFQ Note - I used chatgpt to assist with the syntax here

reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
reviews_df.head()



,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies; creative_works: The Attachmen...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies; persons: Kapur, Shekhar; pers...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies; creative_works: You Can Live ...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies; creative_works: A Tourist's G...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies; persons: Zlotowski, Rebecca; ...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [62]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database

titles = reviews_df['title'].to_list()
print(titles) 

['The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'Pamela, a Love Story', 'In From the Side', 'After Love', 'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley’s Lover', 'The Sound of Christmas', 'The Inspection', 'Bones and All', 'My Policeman', 'About Fate', 'Waiting for Bojangles', 'I Love My Dad', 'A Love Song', 'Alone Together', 'Art of Love', 'The Wheel', 'Thor: Love and Thunder', 'Both Sides of the Blade', 'Fire of Love', 'Love & Gelato', 'Stay Prayed Up', 'Benediction', 'Dinner in America', 'In a New York Minute', 'Anaïs in Love', 'I Love America', 'See You Then', 'La Mami', 'Love After Love', 'Deep Water', 'Lucy and Desi', 'Cyrano', 'The In Between', 'Book of Love', 'Lingui, the Sacred Bonds', 'The Pink Cloud', 'A Journal for Jordan', 'West Side Story'

In [63]:
titles_df = pd.DataFrame(titles, columns=['title'])
print(titles_df)

                                             title
0                           The Attachment Diaries
1    What’s Love Got to Do With It?’ Probably a Lo
2                             You Can Live Forever
3                        A Tourist’s Guide to Love
4                          Other People’s Children
..                                             ...
195                                 The Other Half
196                         The Ottoman Lieutenant
197                                   Love & Taxes
198  Everybody Loves Somebody,’ a Rom-Com With Bit
199              Kedi,’ Rekindling a ‘Love of Life

[200 rows x 1 columns]


### Access The Movie Database API

In [64]:
# Prepare The Movie Database query - search structure is: the url below + SEARCH TERMS + tmdb_key_string

load_dotenv()
tmbd_api_key = os.getenv('TMDB_API_KEY')

url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmbd_api_key


In [65]:
tmdb_movies_list = []

for title in titles:
    search_url = f'https://api.themoviedb.org/3/search/movie?api_key=dcd69a404524d547b711b3fc81aa2f53&query={title}'
    search_response = requests.get(search_url)
    search_data = search_response.json()
    
    try:
        movie_id = search_data['results'][0]['id']
        movie_url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key=dcd69a404524d547b711b3fc81aa2f53'
        movie_response = requests.get(movie_url)
        movie_data = movie_response.json()
        
        genres = [genre['name'] for genre in movie_data['genres']]
        spoken_languages = [lang['english_name'] for lang in movie_data['spoken_languages']]
        production_countries = [country['name'] for country in movie_data['production_countries']]
        
        movie_details = {
            'title': movie_data['title'],
            'original_title': movie_data['original_title'],
            'budget': movie_data['budget'],
            'original_language': movie_data['original_language'],
            'homepage': movie_data.get('homepage', ''),
            'overview': movie_data['overview'],
            'popularity': movie_data['popularity'],
            'runtime': movie_data.get('runtime', 0),
            'revenue': movie_data['revenue'],
            'release_date': movie_data['release_date'],
            'vote_average': movie_data['vote_average'],
            'vote_count': movie_data['vote_count'],
            'genres': genres,
            'spoken_languages': spoken_languages,
            'production_countries': production_countries
        }
        
        tmdb_movies_list.append(movie_details)
        print(f"Movie found: {movie_data['title']}")
    except (IndexError, KeyError) as e:
        print(f"Movie not found: {title}")

# Preview the first five results in JSON format
#print(json.dumps(tmdb_movies_list[:5], indent=4))

# Convert the results to a DataFrame
#tmdb_df = pd.DataFrame(tmdb_movies_list)
#print(tmdb_df.head())


Movie found: The Attachment Diaries
Movie not found: What’s Love Got to Do With It?’ Probably a Lo
Movie found: You Can Live Forever
Movie found: A Tourist's Guide to Love
Movie found: Other People's Children
Movie found: One True Loves
Movie found: The Lost Weekend: A Love Story
Movie found: A Thousand and One
Movie found: Your Place or Mine
Movie found: Love in the Time of Fentanyl
Movie found: Pamela, A Love Story
Movie found: In from the Side
Movie found: After Love
Movie found: Alcarràs
Movie found: Nelly and Monsieur Arnaud
Movie found: Lady Chatterley's Lover
Movie found: The Sound of Christmas
Movie found: The Inspection
Movie found: Bones and All
Movie found: My Policeman
Movie found: About Fate
Movie found: Waiting for Bojangles
Movie found: I Love My Dad
Movie found: A Love Song
Movie found: Alone Together
Movie found: Art of Love
Movie found: 2099: The Soldier Protocol
Movie found: Thor: Love and Thunder
Movie found: Both Sides of the Blade
Movie found: Fire of Love
Movie f

In [67]:
# Preview the first 5 results in JSON format
print(json.dumps(tmdb_movies_list[:5], indent=4))

# Use json.dumps with argument indent=4 to format data






[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 3.871,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

In [68]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",3.871,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",35.245,96,15055,2023-03-24,6.474,38,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",30.992,96,0,2023-04-21,6.287,162,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",12.635,104,84178,2022-09-21,6.818,190,"[Drama, Comedy]","[French, English]",[France]
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,12.922,100,37820,2023-04-07,6.500,75,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,The Last Face,The Last Face,0,en,,"Miguel, a heroic Spanish doctor, puts himself ...",13.138,130,0,2017-01-11,5.600,193,[Drama],"[English, Portuguese]",[United States of America]
150,Lost in Paris,Paris pieds nus,0,fr,,Fiona visits Paris for the first time to assis...,10.091,83,0,2017-01-14,6.022,114,[Comedy],[French],"[Belgium, France]"
151,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,6.289,103,0,2016-12-02,6.300,24,"[Drama, Romance]",[English],[Canada]
152,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",16.043,111,413844,2017-03-10,6.164,247,"[Romance, Drama, War]","[Turkish, English]","[Turkey, United States of America]"


### Merge and Clean the Data for Export

In [69]:
# Merge the New York Times reviews and TMDB DataFrames on title
first_merge = pd.merge(reviews_df, tmdb_df, on="title")
#first_merge.columns
display(first_merge)
#first_merge.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies; creative_works: The Attachmen...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",3.871,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies; creative_works: You Can Live ...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",35.245,96,15055,2023-03-24,6.474,38,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies; persons: Bracey, Luke (1989- ...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,12.922,100,37820,2023-04-07,6.500,75,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs; creat...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,10.718,95,77145,2023-04-13,6.000,2,[Documentary],[English],[United States of America]
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies; creative_works: A Thousand an...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,13.552,116,3395595,2023-03-31,6.871,89,"[Drama, Crime]","[English, Spanish, Portuguese]",[United States of America]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,https://www.nytimes.com/2017/09/14/movies/in-s...,A sheltered young woman enamored of the films ...,The New York Times,subject: Movies; creative_works: In Search of ...,2017-09-14T11:00:24+0000,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,...,"Lucy, a small-town girl from Ohio, discovers t...",4.198,93,0,2017-09-15,5.800,40,"[Drama, Adventure]",[English],[]
124,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies; creative_works: The Last Face...,2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,...,"Miguel, a heroic Spanish doctor, puts himself ...",13.138,130,0,2017-01-11,5.600,193,[Drama],"[English, Portuguese]",[United States of America]
125,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie); subject...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,...,Fiona visits Paris for the first time to assis...,10.091,83,0,2017-01-14,6.022,114,[Comedy],[French],"[Belgium, France]"
126,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies; creative_works: The Other Hal...,2017-03-09T2

In [70]:
first_merge.columns

Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.person',
       'byline.organization', 'title', 'original_title', 'budget',
       'original_language', 'homepage', 'overview', 'popularity', 'runtime',
       'revenue', 'release_date', 'vote_average', 'vote_count', 'genres',
       'spoken_languages', 'production_countries'],
      dtype='object')

In [71]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing

columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]


# Loop through the list of columns to fix

for columns in columns_to_fix:
     # Convert the column to type 'str'
     first_merge[columns] = first_merge[columns].astype(str)
     for char in characters_to_remove:
        first_merge[columns] = first_merge[columns].str.replace(char, "", regex=False)

# Display the fixed DataFrame
display(first_merge.head())


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies; creative_works: The Attachmen...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",3.871,102,0,2021-10-07,3.000,4,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies; creative_works: You Can Live ...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",35.245,96,15055,2023-03-24,6.474,38,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies; persons: Bracey, Luke (1989- ...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,12.922,100,37820,2023-04-07,6.500,75,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs; creat...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,10.718,95,77145,2023-04-13,6.000,2,Documentary,English,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies; creative_works: A Thousand an...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,13.552,116,3395595,2023-03-31,6.871,89,"Drama, Crime","English, Spanish, Portuguese",United States of America


In [73]:
# Drop "byline.person" column

first_merge_del = first_merge

del first_merge_del['byline.person']
first_merge_del.columns


Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.organization', 'title',
       'original_title', 'budget', 'original_language', 'homepage', 'overview',
       'popularity', 'runtime', 'revenue', 'release_date', 'vote_average',
       'vote_count', 'genres', 'spoken_languages', 'production_countries'],
      dtype='object')

In [76]:

drop_dupes = first_merge_del.drop_duplicates()
drop_dupes.reset_index(drop=True)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies; creative_works: The Attachmen...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",3.871,102,0,2021-10-07,3.000,4,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies; creative_works: You Can Live ...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",35.245,96,15055,2023-03-24,6.474,38,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies; persons: Bracey, Luke (1989- ...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,12.922,100,37820,2023-04-07,6.500,75,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs; creat...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,10.718,95,77145,2023-04-13,6.000,2,Documentary,English,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies; creative_works: A Thousand an...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,13.552,116,3395595,2023-03-31,6.871,89,"Drama, Crime","English, Spanish, Portuguese",United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,https://www.nytimes.com/2017/09/14/movies/in-s...,A sheltered young woman enamored of the films ...,The New York Times,subject: Movies; creative_works: In Search of ...,2017-09-14T11:00:24+0000,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,...,"Lucy, a small-town girl from Ohio, discovers t...",4.198,93,0,2017-09-15,5.800,40,"Drama, Adventure",English,
121,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies; creative_works: The Last Face...,2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,...,"Miguel, a heroic Spanish doctor, puts himself ...",13.138,130,0,2017-01-11,5.600,193,Drama,"English, Portuguese",United States of America
122,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie); subject...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,...,Fiona visits Paris for the first time to assis...,10.091,83,0,2017-01-14,6.022,114,Comedy,French,"Belgium, France"
123,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies; creative_works: The Other Hal...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find

In [78]:
# Export data to CSV without the index
drop_dupes.to_csv('C:/Users/Daniel Quinn/Desktop/Bootcamp/Challenge Assignments/data-sourcing-challenge/final_dataframe.csv', index=False, header=True)